In [ ]:
#install.packages(c("ggplot2", "car", "lmtest", "MASS", "nortest", "glmnet", "caret"))
library(ggplot2)# Para gráficos
library(car)# Para diagnósticos de regresión (VIF)
library(lmtest)# Para tests de diagnóstico (bptest)
library(MASS)# Para regresión robusta (rlm)
library(nortest)# Para tests de normalidad
library(glmnet)# Para regresión penalizada (Ridge, Lasso, Elastic Net)
library(caret)# Para facilitar el modelado y la evaluación

Installing packages into ‘/home/alvarezg/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

Warning message:
“package ‘MASS’ is not available for this version of R
‘MASS’ version 7.3-65 is in the repositories but depends on R (>= 4.4.0)
‘MASS’ version 7.3-65 is in the repositories but depends on R (>= 4.6)

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Loading required package: carData

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: Matrix

Loaded glmnet 4.1-10

Loading required package: lattice



In [9]:
# Cargar los datos
datos <- read.csv("StudentsPerformance[1].csv", sep = ",", header = TRUE, stringsAsFactors = FALSE)
head(datos)

,gender,race.ethnicity,parental.level.of.education,lunch,test.preparation.course,math.score,reading.score,writing.score
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
1,female,group B,bachelor's degree,standard,none,72,72,74
2,female,group C,some college,standard,completed,69,90,88
3,female,group B,master's degree,standard,none,90,95,93
4,male,group A,associate's degree,free/reduced,none,47,57,44
5,male,group C,some college,standard,none,76,78,75
6,female,group B,associate's degree,standard,none,71,83,78


In [ ]:
# Cargar los datos y renombrar las columnas
datos <- read.csv("StudentsPerformance[1].csv", sep = ",", header = TRUE, stringsAsFactors = FALSE)
colnames(datos) <- c(
  "gender",
  "race_ethnicity",
  "parent_education",
  "lunch",
  "test_prep",
  "math_score",
  "reading_score",
  "writing_score"
)

# Convertir variables categóricas a factores
datos <- within(datos, {
  gender <- as.factor(gender)
  race_ethnicity <- as.factor(race_ethnicity)
  parent_education <- as.factor(parent_education)
  lunch <- as.factor(lunch)
  test_prep <- as.factor(test_prep)
})